In [12]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/datasets/khanraiyanibnereza/agrivision-bangladesh/AgriVision/data.yaml
/kaggle/input/datasets/khanraiyanibnereza/agrivision-bangladesh/AgriVision/val/papaya_main_dataset__Carica_Insect_Hole/papaya (35).png
/kaggle/input/datasets/khanraiyanibnereza/agrivision-bangladesh/AgriVision/val/papaya_main_dataset__Carica_Insect_Hole/IMG_20251121_155342.png
/kaggle/input/datasets/khanraiyanibnereza/agrivision-bangladesh/AgriVision/val/papaya_main_dataset__Carica_Insect_Hole/_DSC7612.png
/kaggle/input/datasets/khanraiyanibnereza/agrivision-bangladesh/AgriVision/val/papaya_main_dataset__Carica_Insect_Hole/IMG20241110190218.png
/kaggle/input/datasets/khanraiyanibnereza/agrivision-bangladesh/AgriVision/val/papaya_main_dataset__Carica_Insect_Hole/papaya (30).png
/kaggle/input/datasets/khanraiyanibnereza/agrivision-bangladesh/AgriVision/val/papaya_main_dataset__Carica_Insect_Hole/papaya (2).png
/kaggle/input/datasets/khanraiyanibnereza/agrivision-bangladesh/AgriVision/val/papaya_main_data

In [13]:
# =============================================
# Cell 1: Install Ultralytics & basic checks
# =============================================
!pip install --upgrade ultralytics

import ultralytics
ultralytics.checks()  # shows GPU, versions, etc. → screenshot this for report

import os
from pathlib import Path
from ultralytics import YOLO
import matplotlib.pyplot as plt
%matplotlib inline

Ultralytics 8.4.17 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 14913MiB)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 6672.9/8062.4 GB disk)


In [20]:
# Cell 2: Debug paths & fallback yaml – improved version
import os
from pathlib import Path

DATASET_SLUG = "datasets/khanraiyanibnereza/agrivision-bangladesh"  # exact from your URL

# Primary expected path (matches your upload structure)
BASE_DIR = Path("/kaggle/input/datasets/khanraiyanibnereza/agrivision-bangladesh/AgriVision")

# Debug prints – run this to diagnose
print("Full expected BASE_DIR:", BASE_DIR)
print("Does BASE_DIR exist?", BASE_DIR.exists())
print("Contents of BASE_DIR if exists:", os.listdir(BASE_DIR) if BASE_DIR.exists() else "Not found")

print("\nDirect /kaggle/input contents (top level):", os.listdir("/kaggle/input"))
print("Is 'agrivision-bangladesh' in input?", "agrivision-bangladesh" in os.listdir("/kaggle/input"))

# If BASE_DIR doesn't exist, try fallback (sometimes slug has underscores or variations)
if not BASE_DIR.exists():
    print("Trying fallback slug variations...")
    possible_slugs = [DATASET_SLUG, "agrivision_bangladesh", DATASET_SLUG.replace("-", "_")]
    for slug in possible_slugs:
        temp_dir = Path(f"/kaggle/input/{slug}/AgriVision")
        if temp_dir.exists():
            BASE_DIR = temp_dir
            print(f"Found with variation: {slug}")
            break

# Now check subfolders
TRAIN_DIR = BASE_DIR / "train"
print("train exists?", TRAIN_DIR.exists())
if TRAIN_DIR.exists():
    print("Number of class folders in train:", len(os.listdir(TRAIN_DIR)))

# YAML path
YAML_PATH = BASE_DIR / "data.yaml"
if YAML_PATH.exists():
    print("data.yaml found in dataset root – great!")
else:
    print("data.yaml NOT found in dataset – creating fallback in working dir")
    yaml_content = f"""
path: {''}
train: train
val: val
test: test

nc: 57
# names: auto-detected from class folder names
"""
    working_yaml = Path("/kaggle/working/data.yaml")
    with open(working_yaml, "w") as f:
        f.write(yaml_content.strip())
    YAML_PATH = working_yaml
    print("Fallback data.yaml created at:", working_yaml)

Full expected BASE_DIR: /kaggle/input/datasets/khanraiyanibnereza/agrivision-bangladesh/AgriVision
Does BASE_DIR exist? True
Contents of BASE_DIR if exists: ['data.yaml', 'val', 'test', 'train']

Direct /kaggle/input contents (top level): ['datasets']
Is 'agrivision-bangladesh' in input? False
train exists? True
Number of class folders in train: 57
data.yaml found in dataset root – great!


In [22]:
# Training cell – fixed for classification (no yaml needed)
from ultralytics import YOLO

model = YOLO("yolo26n-cls.pt")  # nano classification

results = model.train(
    data=str(BASE_DIR),           # ← this is the key fix: directory root!
    epochs=25,                   # or start with 50 to test
    imgsz=512,
    batch=24,                     # safe start; try 32 if no OOM
    device=0,
    workers=4,
    project="/kaggle/working/runs/train_cls",
    name="57cls_agri_first",
    patience=25,
    optimizer="AdamW",
    lr0=0.01,
    cos_lr=True,
    augment=True,
    hsv_h=0.015, hsv_s=0.7, hsv_v=0.4,
    degrees=15.0,
    translate=0.1, scale=0.5,
    flipud=0.5, fliplr=0.5,
    mosaic=1.0,
    save_period=5,
    plots=True,                   # generates confusion/PR/F1 curves
    exist_ok=True
)

Ultralytics 8.4.17 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 14913MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=True, auto_augment=randaugment, batch=24, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/kaggle/input/datasets/khanraiyanibnereza/agrivision-bangladesh/AgriVision, degrees=15.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=25, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.5, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo26n-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=57cls_agri_first, nbs=64, nms=False, opset=None, optimize=False, optimi

In [23]:
# After training or resume
metrics = model.val()  # on val set
print("Final val top1 accuracy:", metrics.top1)
print("Final val top5 accuracy:", metrics.top5)
print("Full metrics:", metrics.results_dict)

Ultralytics 8.4.17 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 14913MiB)
YOLO26n-cls summary (fused): 47 layers, 1,599,041 parameters, 0 gradients, 3.2 GFLOPs
train: /kaggle/input/datasets/khanraiyanibnereza/agrivision-bangladesh/AgriVision/train... found 8376 images in 57 classes ✅ 
val: /kaggle/input/datasets/khanraiyanibnereza/agrivision-bangladesh/AgriVision/val... found 1068 images in 57 classes ✅ 
test: /kaggle/input/datasets/khanraiyanibnereza/agrivision-bangladesh/AgriVision/test... found 1282 images in 57 classes ✅ 
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 778.6±457.0 MB/s, size: 852.6 KB)
val: Scanning /kaggle/input/datasets/khanraiyanibnereza/agrivision-bangladesh/AgriVision/val... 1055 images, 13 corrupt: 100% ━━━━━━━━━━━━ 1068/1068 1.1Kit/s 1.0ss<0.0s
val: /kaggle/input/datasets/khanraiyanibnereza/agrivision-bangladesh/AgriVision/val/Rice__Healthy_Rice_Leaf/Healthy_rice_leaf (4).jpg: ignoring corrupt image/label: [Errno 30] Read-only file system: '/kaggle

In [27]:
from IPython.display import Image, display
import matplotlib.pyplot as plt
import pandas as pd

run_dir = "/kaggle/working/runs/train_cls/57cls_agri_first_run"  # change if name different

# Confusion matrix
cm = f"{run_dir}/confusion_matrix.png"
if os.path.exists(cm):
    display(Image(cm, width=800))

# Training curves
csv = f"{run_dir}/results.csv"
if os.path.exists(csv):
    df = pd.read_csv(csv)
    df.columns = df.columns.str.strip()
    
    fig, axs = plt.subplots(2, 2, figsize=(14,10))
    
    axs[0,0].plot(df['epoch'], df['train/loss'], label='train loss')
    axs[0,0].plot(df['epoch'], df['val/loss'], label='val loss')
    axs[0,0].legend(); axs[0,0].set_title('Loss'); axs[0,0].grid(True)
    
    axs[0,1].plot(df['epoch'], df['metrics/accuracy_top1/val'], label='val top1 acc')
    axs[0,1].legend(); axs[0,1].set_title('Top-1 Accuracy'); axs[0,1].grid(True)
    
    axs[1,0].plot(df['epoch'], df['metrics/precision/val'], label='precision')
    axs[1,0].plot(df['epoch'], df['metrics/recall/val'], label='recall')
    axs[1,0].legend(); axs[1,0].set_title('Precision/Recall'); axs[1,0].grid(True)
    
    plt.tight_layout()
    plt.show()
    fig.savefig("/kaggle/working/training_curves.png", dpi=300)

In [28]:
!cp /kaggle/working/runs/train_cls/57cls_agri_first_ru/weights/best.pt /kaggle/working/best_classifier.pt

cp: cannot stat '/kaggle/working/runs/train_cls/57cls_agri_first_run/weights/best.pt': No such file or directory
